In [1]:
import requests
import bs4
import re
import pandas as pd
import time
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from requests.adapters import HTTPAdapter, Retry

In [2]:
url = 'https://link.springer.com/journals/a/1'
response = requests.get(url)
soup = bs4.BeautifulSoup(response.content, 'html.parser')

In [3]:
main_url = 'https://link.springer.com'
base_links_list = soup.find('ol', {'class', 'c-atoz-navigation u-mb-24'})
get_link = [link['href'] for link in base_links_list.select('a')]
journal_link_list = [urljoin(str(main_url), str(get_link)) for get_link in get_link]
journal_link_list.insert(0, url)

In [4]:
main_url = 'https://link.springer.com'
base_links_list = soup.find('ol', {'class', 'c-atoz-navigation u-mb-24'})
get_link = [link['href'] for link in base_links_list.select('a')]
journal_link_list = [urljoin(str(main_url), str(get_link)) for get_link in get_link]
journal_link_list.insert(0, url)

list_source_page = []
for i in range(len(journal_link_list)):
    request = requests.get(journal_link_list[i])
    soup = bs4.BeautifulSoup(request.content, 'html.parser')
    source_link_page = soup.find_all('ol', {'class', 'c-pagination-listed__list'})
    
    for link in soup.find_all('a', {'class', 'c-pagination-listed__link'}):
        journal = link.get('href')
        list_source_page.append(journal)

journal_link_extend_link = []     
for j in range(len(list_source_page)):
    rename_url = main_url + list_source_page[j]
    journal_link_extend_link.append(rename_url)

def combine_link_journal(journal_link_list, journal_link_extend_link):
    journal_link_list += journal_link_extend_link
    journal_link_list.sort()

    rerange_link = journal_link_list.pop(0)
    journal_link_list.append(rerange_link)

    return journal_link_list

result_list = combine_link_journal(journal_link_list, journal_link_extend_link)
all_link_journals = list(dict.fromkeys(result_list))

In [128]:
journal_data = []
STT = 0
for link in range(len(all_link_journals)):
    request = requests.get(all_link_journals[link])
    soup = BeautifulSoup(request.content, 'html.parser')
    title_category = soup.find('h1', class_='u-serif').text.strip()
    name_category = soup.find_all('a', class_='c-atoz-list__link')

    if not soup.find('a', class_='c-atoz-list__link'):
        STT+=1
        journal_data.append({
                            'STT': STT,
                            'Danh mục': title_category,
                            'URL danh mục': all_link_journals[link],
                            'Tên tạp chí': '0 có tạp chí'})

    for name_text in name_category:
        STT+=1
        journal_data.append({                            
                            'STT': STT,
                            'Danh mục': title_category,
                            'URL danh mục': all_link_journals[link],
                            'Tên tạp chí': name_text.get_text().strip()})

In [130]:
springer_journals = pd.DataFrame(journal_data, columns = ['STT', 'Danh mục', 'URL danh mục', 'Tên tạp chí'])
springer_journals.to_csv('Crawling Catalougue Springer Link Journals A-Z.csv', index=False)

In [6]:
session = requests.Session()

adapter = HTTPAdapter(max_retries=Retry(total = 5, backoff_factor = 1, allowed_methods = None, status_forcelist=[429, 500, 502, 503, 504]))

session.mount("http://", adapter)
session.mount("https://", adapter)

In [100]:
springer_journals_data = []
STT = 0
for link in all_link_journals:
    request = session.get(link, timeout=(5, 15))
    soup = BeautifulSoup(request.content, 'html.parser')
    href_values = [href_publications.get('href') for href_publications in soup.find_all('a', {'class': 'c-atoz-list__link'})]
    name_categories = soup.find_all('a', class_='c-atoz-list__link')

    for name_category, element in zip(name_categories, href_values):
        try:
            request = session.get(element)
            soup = BeautifulSoup(request.content, 'html.parser')

            #issn
            print_issn_extract = None

            print_issn_dt = soup.find('dt', class_='c-list-description__term', string = 'Print ISSN')
            if print_issn_dt:
                print_issn_extract = print_issn_dt.find_next('dd', class_='c-list-description__details').get_text(strip=True)

            if print_issn_extract is None:
                print_issn_online = soup.find('span', {'itemprop': 'onlineIssn'})
                if print_issn_online:
                    print_issn_extract = print_issn_online.get_text(strip=True)

            if print_issn_extract is None:
                print_issn_li = soup.find_all('li', {'style': 'list-style: none; margin-left: 0'})
                for print_issn in print_issn_li:
                    print_issn_extract = re.search(r'ISSN:\s+(\d{4}-\d{4}) \(print\)', print_issn.get_text(strip=True))
                    if print_issn_extract:
                        print_issn_extract = print_issn_extract.group(1)
                        break
                    else:
                        print_issn_extract = " "
            
            if print_issn_extract is None:
                print_issn_footer = soup.find('p', {'class': 'c-journal-footer__issn'})
                if print_issn_footer:
                    print_issn_extract = re.search(r'\s+(\d{4}-\d{4})', print_issn_footer.get_text(strip=True))
                    if print_issn_extract:
                        print_issn_extract = print_issn_extract.group(1)
                    else:
                        print_issn_extract = " "
                else:
                    print_issn_extract = " "

            #electronic issn
            electronic_issn_extract = None
            
            electronic_issn_dd = soup.find('dd', class_='c-list-description__details u-ma-0')
            if electronic_issn_dd:
                electronic_issn_extract = electronic_issn_dd.get_text(strip=True)

            if electronic_issn_extract is None:
                electronic_issn_li_elements = soup.find_all('li', {'style': 'list-style: none; margin-left: 0'})
                for electronic_issn_li in electronic_issn_li_elements:
                    electronic_issn_extract = re.search(r'ISSN:\s+(\d{4}-\d{4}) \(electronic\)', electronic_issn_li.get_text(strip=True))
                    if electronic_issn_extract:
                        electronic_issn_extract = electronic_issn_extract.group(1)
                        break
                    else:
                        electronic_issn_extract = " "

            if electronic_issn_extract is None:
                electronic_issn_li_list = soup.find('li', class_='c-list-group__item', string = re.compile(r'ISSN:\s+\d{4}-\d{4} \(electronic\)'))
                if electronic_issn_li_list:
                    electronic_issn_extract = re.search(r'ISSN:\s+(\d{4}-\d{4}) \(electronic\)', electronic_issn_li_list.get_text(strip=True))
                    if electronic_issn_extract:
                        electronic_issn_extract = electronic_issn_extract.group(1)
                    else:
                        electronic_issn_extract = " "
                else:
                    electronic_issn_extract = " "

            #impact factor
            impact_factor_extract = None
            impact_factor_compare = r'^\d\.\d \- 2-year Impact Factor'

            impact_factor_2y_dd = soup.find('dd', attrs={'data-test': 'impact-factor-value'})
            if impact_factor_2y_dd:
                impact_factor_text = impact_factor_2y_dd.get_text(strip=True)
                impact_factor_extract = re.search(r'^([\d.]+)', impact_factor_text)
                if impact_factor_extract:
                    impact_factor_extract = impact_factor_extract.group(1)
                        
            if impact_factor_extract is None:
                impact_factor_span = soup.find('span', class_='impact-factor__value')
                if impact_factor_span:
                    impact_factor_extract = impact_factor_span.get_text(strip=True)

            if impact_factor_extract is None:
                impact_factor_2y_strong = soup.find('strong', string='2022 Citation Impact')
                next_sibling = impact_factor_2y_strong.find_next_sibling('br') if impact_factor_2y_strong else None
                impact_factor_extract = re.search(r'^\d\.\d \- \d\-\w\w\w\w \w\w\w\w\w\w \w\w\w\w\w\w$', next_sibling.next_sibling.strip()) if next_sibling and next_sibling.next_sibling else None
                if impact_factor_extract:
                    impact_factor = impact_factor_extract.group()
                    if impact_factor and re.match(impact_factor_compare, impact_factor):
                        impact_factor_extract = re.match(r'^\d\.\d', impact_factor).group()
                    else:
                        impact_factor_extract = " "
                else:
                    impact_factor_extract = " "


            STT+=1
            springer_journals_data.append({
                                        'STT': STT,
                                        'Tên tạp chí': name_category.get_text().strip(),
                                        'ISSN': print_issn_extract,
                                        'eISSN': electronic_issn_extract,
                                        'Impact Factor': impact_factor_extract})
            
        except requests.exceptions.RequestException as err:
            print("Request error:", err)
            time.sleep(5)

In [110]:
springer_journals_data_book = pd.DataFrame(springer_journals_data, columns = ['STT', 'Tên tạp chí', 'ISSN', 'eISSN', 'Impact Factor'])
springer_journals_data_book.to_csv('Crawling Springer Journal A-Z.csv', index=False)